In [4]:
from qcutils.credential import load_provider
from qcutils.benchmarks import QASMBenchmark
from qcutils.pulse import CircAmpTimeSeries, BasisPulse
from qcutils.pulse import CircAmpTimeSeries
from qcutils.pulse.amptimeseries.metric import dist, norm
from qcutils.circuit.library import deutsch_jozsa, bernstein_vazirani, grover

from qiskit import transpile, schedule


provider = load_provider()
backend_list = provider.backends(simulator=False)

bm = QASMBenchmark(r"E:\Research\QASMBench", "small", num_qubits_list=list(range(6)), remove_final_measurements=True, do_transpile=False)

In [9]:
# save the current basis pulse library

for backend in backend_list:
    bp = BasisPulse(backend)
    bp.save_basis_amp_time_series_list(f"./data/basis_pulse_{backend.configuration().backend_name}.pickle")

In [2]:
def min_normalized_circ_total_dist_in_backend_list(circ, backend_list, **trasnpile_args):
    import numpy as np
    min_normalized_circ_total_dist = float("inf")
    for i, backend in enumerate(backend_list[:-1]):
        total_a = CircAmpTimeSeries.from_circ(transpile(circ, backend, **trasnpile_args), backend).total()
        total_b_list = [CircAmpTimeSeries.from_circ(transpile(circ, b, **trasnpile_args), b).total() for b in backend_list[i+1:]]
        dist_list = [dist(total_a, total_b) for total_b in total_b_list]
        min_dist_idx = np.argmin(dist_list)
        curr_min = dist_list[min_dist_idx]
        normalized_min_a = curr_min / norm(total_a)
        normalized_min_b = curr_min / norm(total_b_list[min_dist_idx])
        if normalized_min_a < min_normalized_circ_total_dist or normalized_min_b < min_normalized_circ_total_dist:
            if normalized_min_a < normalized_min_b:
                min_idx_pair = (i, i + 1 + min_dist_idx)
                min_normalized_circ_total_dist = normalized_min_a
            else:
                min_idx_pair = (i + 1 + min_dist_idx, i)
                min_normalized_circ_total_dist = normalized_min_b
    return min_normalized_circ_total_dist, min_idx_pair


def result(bm, backend_list, **transpile_args):
    print("Benchmark\t\tMin CD (QP)\tMin IDX Pair")

    for circ_name in bm.circ_name_list:
        try:
            min_dist, min_idx_pair = min_normalized_circ_total_dist_in_backend_list(bm.get(circ_name), backend_list, **transpile_args)
            print(f"{circ_name:<24}{min_dist:.3f}\t\t{min_idx_pair}")
        except:
            continue

In [8]:
result(bm, backend_list, seed_transpiler=0)

Benchmark		Min CD (QP)	Min IDX Pair
deutsch_n2              0.116		(2, 3)
dnn_n2                  0.116		(2, 3)
grover_n2               0.116		(2, 3)
iswap_n2                0.116		(2, 3)
quantumwalks_n2         0.117		(2, 3)
basis_change_n3         0.889		(0, 6)
fredkin_n3              0.851		(4, 6)
linearsolver_n3         0.688		(3, 2)
qaoa_n3                 0.831		(4, 6)
teleportation_n3        0.580		(3, 2)
toffoli_n3              0.869		(1, 6)
wstate_n3               0.849		(4, 5)
adder_n4                0.743		(7, 8)
basis_trotter_n4        0.865		(4, 5)
bell_n4                 0.793		(2, 5)
cat_state_n4            0.776		(4, 5)
hs4_n4                  0.801		(5, 6)
inverseqft_n4           0.002		(8, 6)
qft_n4                  0.846		(4, 6)
qrng_n4                 0.002		(8, 6)
variational_n4          0.783		(3, 8)
vqe_n4                  0.808		(2, 8)
vqe_uccsd_n4            0.919		(5, 6)
error_correctiond3_n5   0.930		(4, 5)
lpn_n5                  0.315		(0, 4)
pea_n5        